In [1]:
# load dependencies'
import concurrent.futures
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape
import osmnx as ox
import networkx as nx
import numpy as np
import requests
import json
import matplotlib.pyplot as plt
from urllib.parse import urljoin
from shapely.geometry import Point, LineString, Polygon
import pyproj 
import mm_utils

In [2]:
# this function outputs the circle of radius 200m centered at the current location. 
def circle_area(curr_loc):
    
    x = curr_loc['geometry'].iloc[0].x
    y = curr_loc['geometry'].iloc[0].y

    p1 = Point((x,y))
    df = pd.DataFrame({'Attribute' : ['name1']})
    circle_ragion = gpd.GeoDataFrame(df, geometry = [p1], crs = "EPSG:32631")
    circle_ragion['geometry'] = circle_ragion.geometry.buffer(200)

    return circle_ragion

In [13]:
trajectory_data = pd.read_pickle('gdf_utm.pkl')
nodes_data = pd.read_pickle('nodes_utm.pkl')
edges_data = pd.read_pickle('edges_utm.pkl')

In [14]:
trajectory_data.crs

<Projected CRS: EPSG:32631>
Name: WGS 84 / UTM zone 31N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 0°E and 6°E, northern hemisphere between equator and 84°N, onshore and offshore. Algeria. Andorra. Belgium. Benin. Burkina Faso. Denmark - North Sea. France. Germany - North Sea. Ghana. Luxembourg. Mali. Netherlands. Niger. Nigeria. Norway. Spain. Togo. United Kingdom (UK) - North Sea.
- bounds: (0.0, 0.0, 6.0, 84.0)
Coordinate Operation:
- name: UTM zone 31N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [18]:
a = trajectory_data.iloc[[5]]

In [19]:
a.crs

<Projected CRS: EPSG:32631>
Name: WGS 84 / UTM zone 31N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 0°E and 6°E, northern hemisphere between equator and 84°N, onshore and offshore. Algeria. Andorra. Belgium. Benin. Burkina Faso. Denmark - North Sea. France. Germany - North Sea. Ghana. Luxembourg. Mali. Netherlands. Niger. Nigeria. Norway. Spain. Togo. United Kingdom (UK) - North Sea.
- bounds: (0.0, 0.0, 6.0, 84.0)
Coordinate Operation:
- name: UTM zone 31N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [15]:
iter = 20

curr_loc = trajectory_data.iloc[iter].to_frame().T

circle_ragion = circle_area(curr_loc)

intersects = gpd.sjoin(circle_ragion, edges_data, predicate='intersects')
# contains_nodes = gpd.sjoin(circle_ragion, nodes_utm, predicate='contains')

inter_index = intersects[['index_right0', 'index_right1', 'index_right2']].values.tolist()
intersects_edges = edges_data.loc[map(tuple, inter_index)]

unique_intersects = intersects_edges.loc[intersects_edges['osmid'].drop_duplicates().index]
temp = unique_intersects.reset_index()
list = pd.concat([temp['u'], temp['v']])
list = list.to_list()
temp2 = np.unique(list, return_counts=True) 

number_junctions = np.count_nonzero(temp2[1] >= 3)

total_length = 0
for i in range(len(contains)):
    if contains.iloc[i]['oneway'] == False:
        total_length += contains.iloc[i]['length']/2
    else:
        total_length += contains.iloc[i]['length']

# km to m
total_length /= 1000

map_area = number_junctions / total_length
 

NameError: name 'contains' is not defined

In [31]:
map_area

0.8308546170591073

In [12]:
%matplotlib tk
# This is how we  visualize edges and error bound 
#print(edges)
print(type(edges_data))

# find which edges is selected at time point
# find index of the edge id
# loc = np.where(edges_data["str_id"] == conc(edge_link[0]))

# find the last position for IMP
# poly_1 = err_polygon(curr_pos_list[iter - 1], err_size)
poly_2 = circle_area(curr_loc)

#Save selected edge 
# answer_loc = edges_data.iloc[loc]

# plotting edges and starting point together 
f, ax = plt.subplots()

# location for all point
#locs_utm.plot(ax=ax)
point_locs = trajectory_data['geometry'].to_frame()
point_locs.iloc[0:iter, :].plot(ax = ax)

#current location versus edges
#curr_loc.plot(ax=ax)

#err coord 
# better if we just take location at the last and use error bound function 
poly_2.plot(ax=ax, facecolor="none")
nodes_data.plot(ax = ax, color = "Green")

# this plot all the road system 
edges_data.plot(ax=ax)

# this plot the selected edge at time point 
# answer_loc.plot(ax=ax, cmap = "Reds")

#print(intersects['index_right'])

<class 'geopandas.geodataframe.GeoDataFrame'>


<Axes: >